In [ ]:
import heamy

In [ ]:
from heamy.dataset import Dataset
from heamy.estimator import Regressor, Classifier
from heamy.pipeline import ModelsPipeline

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline
#from rgf.sklearn import RGFClassifier
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split, cross_val_score
import gc
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import  StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
#from glmnet import LogitNet

In [ ]:
def get_dataset(datatype):
    train = pd.read_hdf('train.h5', key='train').reset_index(drop=True)
    train1_new_label = pd.read_hdf('new_label.h5', key='train1_new_label')
    train2_new_label = pd.read_hdf('new_label.h5', key='train2_new_label')
    train_new_label = pd.concat([train1_new_label, train2_new_label], axis=0, ignore_index=True)
    del train_new_label['user_id']
    train = pd.concat([train, train_new_label], axis=1)
    X_test = pd.read_hdf('test.h5', key='test')
    X_train = train
    uid_tr = X_train['user_id']
    uid_te = X_test['user_id']
    del X_train['user_id'], X_test['user_id']
    del train, train_new_label
    gc.collect()
    if datatype=='lgb':
        print('final shape:', X_train.shape, X_test.shape,)
        return X_train, X_test, uid_te
    if datatype=='NN':
        cate_col = ['register_type']
        cont_col = [i for i in list(X_test) if i not in cate_col]
        X_all_cont = pd.concat([X_train[cont_col], X_test[cont_col]], axis=0).reset_index(drop=True)
        scaler = StandardScaler(with_mean=False )
        scaler.fit(X_all_cont)
        X_train_cont = pd.DataFrame(scaler.transform(X_train[cont_col]), columns=cont_col)
        X_test_cont = pd.DataFrame(scaler.transform(X_test[cont_col]), columns=cont_col)
        X_train_cont['register_type'] = X_train['register_type']
        X_test_cont['register_type'] = X_test['register_type']
        for i in ['label', 'fir_cre_day2final', 'fir_lau_day2final', 'fir_act_day2final', 'future_num']:
            X_train_cont[i] = X_train[i]
        X_train = pd.get_dummies(X_train_cont, columns=['register_type'])
        X_test = pd.get_dummies(X_test_cont, columns=['register_type'])
        print('final shape:', X_train.shape, X_test.shape)
        return X_train, X_test, uid_te

In [ ]:
X_train_lgb, X_test_lgb, uid_te = get_dataset('lgb')
#X_train_NN, X_test_NN, uid_te = get_dataset('NN')
#X_train_NNreg, X_test_NNreg, _= get_dataset('NNreg')

In [ ]:
def targetASfea4lgb(X_train, X_test,target, all_target,  tartype):   
    if tartype == 'binary':
        X_train, Y_train = X_train.drop(all_target, axis=1), X_train[target]
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(X_train, Y_train)
        train_preds = np.zeros(shape=(len(X_train), 1))
        test_preds = np.zeros(shape=(len(X_test), 1))
        for i , (tr_ix, va_ix) in enumerate(skf):
            x_tr, y_tr, x_va, y_va = X_train.iloc[tr_ix, :], Y_train.iloc[tr_ix], X_train.iloc[va_ix, :], Y_train.iloc[va_ix]
            gbm = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, 
                                learning_rate=0.06, 
                                n_estimators=5000,
                                objective='binary', min_split_gain=0.0,
                                min_child_weight=25,
                                min_child_samples=10,
                                subsample=0.8, subsample_freq=1,
                                colsample_bytree=0.6, metric='auc',
                                reg_alpha=0.0, reg_lambda=0.0,
                                random_state=1, n_jobs=-1, silent=False) 
            lgb_tr = lgb.Dataset(x_tr, y_tr,  categorical_feature=['register_type'])
            lgb_va = lgb.Dataset(x_va, y_va, reference=lgb_tr, categorical_feature=['register_type'])
            model = lgb.train(gbm.get_params(), lgb_tr, early_stopping_rounds=50, valid_sets=lgb_va, verbose_eval=100)
            train_preds[va_ix, 0] += model.predict(x_va)
            test_preds[:, 0] += model.predict(X_test)
        test_preds = test_preds / 10
        train_fea = pd.DataFrame(train_preds, columns= ['lgbpred_' + target])
        test_fea = pd.DataFrame(test_preds, columns= ['lgbpred_' + target])
    if tartype == 'multi':
        le = LabelEncoder()
        X_train[target] = le.fit_transform(X_train[target])
        X_train, Y_train = X_train.drop(all_target, axis=1), X_train[target]
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(X_train, Y_train)
        #X_test
        numclass = len(np.unique(Y_train))
        train_preds = np.zeros(shape=(len(X_train), numclass))
        test_preds = np.zeros(shape=(len(X_test), numclass))
        for i , (tr_ix, va_ix) in enumerate(skf):
            x_tr, y_tr, x_va, y_va = X_train.iloc[tr_ix, :], Y_train.iloc[tr_ix], X_train.iloc[va_ix, :], Y_train.iloc[va_ix]
            gbm = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, 
                                learning_rate=0.06, 
                                n_estimators=3000,
                                objective='multiclass', min_split_gain=0.0,
                                num_class=numclass,
                                min_child_weight=25,
                                min_child_samples=10,
                                subsample=0.8, subsample_freq=1,
                                colsample_bytree=0.6, metric='multi_logloss',
                                reg_alpha=0.0, reg_lambda=0.0,
                                random_state=1, n_jobs=-1, silent=False) 
            lgb_tr = lgb.Dataset(x_tr, y_tr,  categorical_feature=['register_type'])
            lgb_va = lgb.Dataset(x_va, y_va, reference=lgb_tr, categorical_feature=['register_type'])
            model = lgb.train(gbm.get_params(), lgb_tr, early_stopping_rounds=50, valid_sets=lgb_va, verbose_eval=100)
            train_preds[va_ix, :] += model.predict(x_va)
            test_preds[:, :] += model.predict(X_test)
        test_preds = test_preds / 10
        train_fea = pd.DataFrame(train_preds, columns= ['lgbpred_' + target + str(i) for i in range(numclass)])
        test_fea = pd.DataFrame(test_preds, columns= ['lgbpred_' + target + str(i) for i in range(numclass)])
    if tartype == 'reg':
        kf = KFold(n_splits=10, shuffle=True, random_state=1).split(X_train)
        gbm = lgb.LGBMRegressor(boosting_type='gbdt', num_leaves=31, max_depth=-1, 
                                learning_rate=0.06, 
                                n_estimators=5000,
                                objective='regression_l2', min_split_gain=0.0,
                                min_child_weight=25,
                                min_child_samples=10,
                                subsample=0.8, subsample_freq=1,
                                colsample_bytree=0.6,
                                reg_alpha=0.0, reg_lambda=0.0,  metric='l2_root',
                                random_state=1, n_jobs=-1, silent=False)  
        X_train, Y_train = X_train.drop(all_target, axis=1), X_train[target]
        #X_test
        train_preds = np.zeros(shape=(len(X_train), 1))
        test_preds = np.zeros(shape=(len(X_test), 1))
        for i , (tr_ix, va_ix) in enumerate(kf):
            x_tr, y_tr, x_va, y_va = X_train.iloc[tr_ix, :], Y_train.iloc[tr_ix], X_train.iloc[va_ix, :], Y_train.iloc[va_ix]
            lgb_tr = lgb.Dataset(x_tr, y_tr, categorical_feature=['register_type'])
            lgb_va = lgb.Dataset(x_va, y_va, reference=lgb_tr, categorical_feature=['register_type'])
            model = lgb.train(gbm.get_params(), lgb_tr, early_stopping_rounds=50, valid_sets=lgb_va, verbose_eval=100)
            train_preds[va_ix, 0] += model.predict(x_va).ravel()
            test_preds[:, 0] += model.predict(X_test).ravel()
        test_preds = test_preds / 10
        train_fea = pd.DataFrame(train_preds, columns= ['lgbpred_' + target])
        test_fea = pd.DataFrame(test_preds, columns= ['lgbpred_' + target])
    return train_fea, test_fea

In [ ]:
leak_multi = [i for i in X_train_lgb.columns if 'fir' in i]
leak_cont = ['future_num']
all_target = leak_multi + leak_cont + ['label']

In [ ]:
ls -lh 

In [ ]:
%%time
train_lgb_label, test_lgb_label = targetASfea4lgb(X_train_lgb,X_test_lgb, 'label',  all_target, 'binary') 
train_lgb_label.to_hdf('../input/leak.h5', key='train_lgb_label')
test_lgb_label.to_hdf('../input/leak.h5', key='test_lgb_label')
print('label done')
train_lgb_leak4, test_lgb_leak4 = targetASfea4lgb(X_train_lgb,X_test_lgb, leak_cont[0],  all_target, 'reg') 
train_lgb_leak4.to_hdf('../input/leak.h5', key='train_lgb_leak4')
test_lgb_leak4.to_hdf('../input/leak.h5', key='test_lgb_leak4')
print('4 done')
train_lgb_leak1, test_lgb_leak1 = targetASfea4lgb(X_train_lgb,X_test_lgb, leak_multi[0],  all_target, 'multi')
train_lgb_leak1.to_hdf('../input/leak.h5', key='train_lgb_leak1')
test_lgb_leak1.to_hdf('../input/leak.h5', key='test_lgb_leak1')
print('1 done')
train_lgb_leak2, test_lgb_leak2 = targetASfea4lgb(X_train_lgb,X_test_lgb, leak_multi[1],  all_target, 'multi') 
train_lgb_leak2.to_hdf('../input/leak.h5', key='train_lgb_leak2')
test_lgb_leak2.to_hdf('../input/leak.h5', key='test_lgb_leak2')
print('2 done')
train_lgb_leak3, test_lgb_leak3 = targetASfea4lgb(X_train_lgb,X_test_lgb, leak_multi[2],  all_target, 'multi')
train_lgb_leak3.to_hdf('../input/leak.h5', key='train_lgb_leak3')
test_lgb_leak3.to_hdf('../input/leak.h5', key='test_lgb_leak3')
print('3 done')

train_lgb_label = pd.read_hdf('../input/leak.h5', key='train_lgb_label')
test_lgb_label = pd.read_hdf('../input/leak.h5', key='test_lgb_label')

train_lgb_leak4 = pd.read_hdf('../input/leak.h5', key='train_lgb_leak4')
test_lgb_leak4 = pd.read_hdf('../input/leak.h5', key='test_lgb_leak4')

train_lgb_leak1 = pd.read_hdf('../input/leak.h5', key='train_lgb_leak1')
test_lgb_leak1 = pd.read_hdf('../input/leak.h5', key='test_lgb_leak1')

train_lgb_leak2 = pd.read_hdf('../input/leak.h5', key='train_lgb_leak2')
test_lgb_leak2 = pd.read_hdf('../input/leak.h5', key='test_lgb_leak2')

train_lgb_leak3 = pd.read_hdf('../input/leak.h5', key='train_lgb_leak3')
test_lgb_leak3 = pd.read_hdf('../input/leak.h5', key='test_lgb_leak3')

lgbtrain_pred_fea = pd.concat([train_lgb_label, train_lgb_leak1, train_lgb_leak2, train_lgb_leak3, train_lgb_leak4], axis=1)
lgbtest_pred_fea = pd.concat([test_lgb_label, test_lgb_leak1, test_lgb_leak2, test_lgb_leak3, test_lgb_leak4], axis=1)


lgbtrain_pred_fea = lgbtrain_pred_fea.apply(pd.to_numeric, downcast='float')
lgbtest_pred_fea = lgbtest_pred_fea.apply(pd.to_numeric, downcast='float')
lgbtrain_pred_fea.to_hdf('pred_fea.h5', key='lgbtrain_pred_fea', complevel=9)
lgbtest_pred_fea.to_hdf('pred_fea.h5', key='lgbtest_pred_fea', complevel=9)

In [ ]:
#以下未实现，效果不佳
"""from keras import backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

def NN_model(X_train, num_per_layer,  tartype, drop_per_layer=[.8, .6, .4]):
    model = Sequential()
    
    model.add(Dense(num_per_layer[0], input_dim = X_train.shape[1], init = 'he_normal'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Dropout(drop_per_layer[0]))
        
    model.add(Dense(num_per_layer[1], init = 'he_normal'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())    
    model.add(Dropout(drop_per_layer[1]))
    
    model.add(Dense(num_per_layer[2], init = 'he_normal'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())    
    model.add(Dropout(drop_per_layer[2]))
    
    if tartype == 'binary':
        model.add(Dense(1, init = 'he_normal', activation='sigmoid'))
        model.compile(loss='binary_crossentropy',
                      optimizer='adam')
    if tartype == 'reg':
        model.add(Dense(1))
        model.compile(optimizer='rmsprop', loss = root_mean_squared_error , metrics=['mae'])
    if tartype == 'multi':
        model.add(Dense(8, activation='softmax'))
        model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return model

def targetASfea4NN(X_train, X_test,target, all_target,  tartype):   
    if tartype == 'binary':
        X_train, Y_train = X_train.drop(all_target, axis=1), X_train[target]
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(X_train, Y_train)
        train_preds = np.zeros(shape=(len(X_train), 1))
        test_preds = np.zeros(shape=(len(X_test), 1))
        for i , (tr_ix, va_ix) in enumerate(skf):
            x_tr, y_tr, x_va, y_va = X_train.iloc[tr_ix, :], Y_train.iloc[tr_ix], X_train.iloc[va_ix, :], Y_train.iloc[va_ix]
            model = NN_model(x_tr, [96,64,32], 'binary')
            model.fit(x_tr, y_tr,epochs=100, batch_size=4096,
                       validation_data=(x_va, y_va),
                    callbacks= [EarlyStopping(monitor='val_loss',patience=3, verbose=30) ], verbose = 30)
            train_preds[va_ix, 0] += model.predict(x_va).ravel()
            test_preds[:, 0] += model.predict(X_test).ravel()
        test_preds = test_preds / 5
        train_fea = pd.DataFrame(train_preds, columns= ['pred_' + target])
        test_fea = pd.DataFrame(test_preds, columns= ['pred_' + target])
    if tartype == 'multi':
        le = LabelEncoder()
        X_train[target] = le.fit_transform(X_train[target])
        X_train, Y_train = X_train.drop(all_target, axis=1), X_train[target]
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(X_train, Y_train)
        numclass = len(np.unique(Y_train))
        train_preds = np.zeros(shape=(len(X_train), numclass))
        test_preds = np.zeros(shape=(len(X_test), numclass))
        for i , (tr_ix, va_ix) in enumerate(skf):
            x_tr, y_tr, x_va, y_va = X_train.iloc[tr_ix, :], Y_train.iloc[tr_ix], X_train.iloc[va_ix, :], Y_train.iloc[va_ix]
            y_tr = np.array(pd.get_dummies(y_tr))
            y_va = np.array(pd.get_dummies(y_va))
            model = NN_model(x_tr, [96,64,32], 'multi')
            model.fit(x_tr, y_tr,epochs=100, batch_size=4096,
                       validation_data=(x_va, y_va),
                    callbacks= [EarlyStopping(monitor='val_loss',patience=3, verbose=30) ], verbose = 30)
            train_preds[va_ix, :] += model.predict(x_va)
            test_preds[:, :] += model.predict(X_test)
        test_preds = test_preds / 5
        train_fea = pd.DataFrame(train_preds, columns= ['pred_' + target + str(i) for i in range(numclass)])
        test_fea = pd.DataFrame(test_preds, columns= ['pred_' + target + str(i) for i in range(numclass)])
    if tartype == 'reg':
        kf = KFold(n_splits=10, shuffle=True, random_state=1).split(X_train)
        X_train, Y_train = X_train.drop(all_target, axis=1), X_train[target]
        #X_test
        train_preds = np.zeros(shape=(len(X_train), 1))
        test_preds = np.zeros(shape=(len(X_test), 1))
        for i , (tr_ix, va_ix) in enumerate(kf):
            x_tr, y_tr, x_va, y_va = X_train.iloc[tr_ix, :], Y_train.iloc[tr_ix], X_train.iloc[va_ix, :], Y_train.iloc[va_ix]
            model = NN_model(x_tr, [96,64,32], 'reg', drop_per_layer=[.5, .25, .15],)
            model.fit(x_tr, y_tr,epochs=100, batch_size=4096,
                       validation_data=(x_va, y_va),
                    callbacks= [EarlyStopping(monitor='val_loss',patience=3, verbose=2) ], verbose = 2)
            train_preds[va_ix, 0] += model.predict(x_va).ravel()
            test_preds[:, 0] += model.predict(X_test).ravel()
        test_preds = test_preds/5
        train_fea = pd.DataFrame(train_preds, columns= ['pred_' + target])
        test_fea = pd.DataFrame(test_preds, columns= ['pred_' + target])
    return train_fea, test_fea

# 16 .45
train_NN_label, test_NN_label = targetASfea4NN(X_train_NN,X_test_NN, 'label',  all_target, 'binary') 
train_NN_leak4, test_NN_leak4 = targetASfea4NN(X_train_NN,X_test_NN, leak_cont[0],  all_target, 'reg') 
train_NN_leak1, test_NN_leak1 = targetASfea4NN(X_train_NN,X_test_NN, leak_multi[0],  all_target, 'multi') 
train_NN_leak2, test_NN_leak2 = targetASfea4NN(X_train_NN,X_test_NN, leak_multi[1],  all_target, 'multi') 
train_NN_leak3, test_NN_leak3 = targetASfea4NN(X_train_NN,X_test_NN, leak_multi[2],  all_target, 'multi') 

NNtrain_pred_fea = pd.concat([train_NN_label, train_NN_leak1, train_NN_leak2, train_NN_leak3, train_NN_leak4], axis=1)
NNtest_pred_fea = pd.concat([test_NN_label, test_NN_leak1, test_NN_leak2, test_NN_leak3, test_NN_leak4], axis=1)
NNtrain_pred_fea.to_hdf('../input/pred_fea.h5', key='NNtrain_pred_fea')
NNtest_pred_fea.to_hdf('../input/pred_fea.h5', key='NNtest_pred_fea')"""